In [42]:
"""

https://www.kaggle.com/osciiart/advanced-basketball-analytics-with-ensembling
"""

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# %matplotlib inline
from math import pi
# import seaborn as sns
import time

from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV, train_test_split, StratifiedKFold, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn import preprocessing, metrics,ensemble, model_selection
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
# from xgboost.sklearn import XGBClassifier
from sklearn.metrics import accuracy_score, roc_curve, auc, classification_report, confusion_matrix

pd.set_option('display.max_columns', 999)
pd.options.display.float_format = '{:.6f}'.format

start_time = time.time()

In [43]:
#standard files

#df_tourney = pd.read_csv('NCAATourneyCompactResults.csv')
#df_season = pd.read_csv('RegularSeasonDetailedResults.csv')
#df_teams = pd.read_csv('Teams.csv')
#df_seeds = pd.read_csv('NCAATourneySeeds.csv')
#df_conferences = pd.read_csv('Conferences.csv')
#df_rankings = pd.read_csv('MasseyOrdinals.csv')
#df_sample_sub = pd.read_csv('SampleSubmissionStage1.csv')
#df_team_conferences = pd.read_csv('Teamconferences.csv')
#df_ConferenceTourneyGames = pd.read_csv('ConferenceTourneyGames.csv')

#my custom file


#df_tourney_experience = pd.read_csv('tourney_experience_senior_class.csv')

# Kaggle locations

df_tourney = pd.read_csv('../input/NCAATourneyCompactResults.csv')
df_season = pd.read_csv('../input/RegularSeasonDetailedResults_Prelim2018.csv') # TODO update
df_teams = pd.read_csv('../input/Teams.csv')
df_seeds = pd.read_csv('../input/NCAATourneySeeds_SampleTourney2018.csv') # TODO update
df_conferences = pd.read_csv('../input/Conferences.csv')
df_rankings = pd.read_csv('../input/MasseyOrdinals_Prelim2018.csv') # TODO update
df_sample_sub = pd.read_csv('../input/SampleSubmissionStage2_SampleTourney2018.csv') # TODO update
df_team_conferences = pd.read_csv('../input/Teamconferences.csv')

#private data file

df_tourney_experience = pd.read_csv('../additional/experience.csv')

In [44]:
#Calculate Winning/losing Team Possesion Feature

#https://www.nbastuffer.com/analytics101/possession/


wPos = df_season.apply(lambda row: 0.96*(row.WFGA + row.WTO + 0.44*row.WFTA - row.WOR), axis=1)
lPos = df_season.apply(lambda row: 0.96*(row.LFGA + row.LTO + 0.44*row.LFTA - row.LOR), axis=1)

#two teams use almost the same number of possessions in a game
#(plus/minus one or two - depending on how quarters end)
#so let's just take the average

df_season['Possesions'] = (wPos+lPos)/2

In [45]:
df_season.head(5)

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,WFGM3,WFGA3,WFTM,WFTA,WOR,WDR,WAst,WTO,WStl,WBlk,WPF,LFGM,LFGA,LFGM3,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF,Possesions
0,2003,10,1104,68,1328,62,N,0,27,58,3,14,11,18,14,24,13,23,7,1,22,22,53,2,10,16,22,10,22,8,18,9,2,20,69.888000
1,2003,10,1272,70,1393,63,N,0,26,62,8,20,10,19,15,28,16,13,4,4,18,24,67,6,24,9,20,20,25,7,12,8,6,16,65.356800
2,2003,11,1266,73,1437,61,N,0,24,58,8,18,17,29,17,26,15,10,5,2,25,22,73,3,26,14,23,31,22,9,12,2,5,23,61.382400
3,2003,11,1296,56,1457,50,N,0,18,38,3,9,17,31,6,19,11,12,14,2,18,18,49,6,22,8,15,17,20,9,19,4,3,23,55.315200
4,2003,11,1400,77,1208,71,N,0,30,61,6,14,11,13,17,22,12,14,4,4,20,24,62,6,16,17,27,21,15,12,10,7,1,14,60.768000


In [46]:
#Name Player Impact Estimate Definition PIE measures a player's overall statistical contribution
#against the total statistics in games they play in. PIE yields results which are
#comparable to other advanced statistics (e.g. PER) using a simple formula.
#Formula (PTS + FGM + FTM - FGA - FTA + DREB + (.5 * OREB) + AST + STL + (.5 * BLK) - PF - TO)
# / (GmPTS + GmFGM + GmFTM - GmFGA - GmFTA + GmDREB + (.5 * GmOREB) + GmAST + GmSTL + (.5 * GmBLK) - GmPF - GmTO)

#We will use this to measure Team Skill

wtmp = df_season.apply(lambda row: row.WScore + row.WFGM + row.WFTM - row.WFGA - row.WFTA + row.WDR + 0.5*row.WOR + row.WAst +row.WStl + 0.5*row.WBlk - row.WPF - row.WTO, axis=1)
ltmp = df_season.apply(lambda row: row.LScore + row.LFGM + row.LFTM - row.LFGA - row.LFTA + row.LDR + 0.5*row.LOR + row.LAst +row.LStl + 0.5*row.LBlk - row.LPF - row.LTO, axis=1) 

df_season['WPIE'] = wtmp/(wtmp + ltmp)
df_season['LPIE'] = ltmp/(wtmp + ltmp)

In [47]:
#Four factors statistic from the NBA

#https://www.nbastuffer.com/analytics101/four-factors/


#Effective Field Goal Percentage=(Field Goals Made) + 0.5*3P Field Goals Made))/(Field Goal Attempts)
#you have to put the ball in the bucket eventually

df_season['WeFGP'] = df_season.apply(lambda row:(row.WFGM + 0.5 * row.WFGM3) / row.WFGA, axis=1)      
df_season['LeFGP'] = df_season.apply(lambda row:(row.LFGM + 0.5 * row.LFGM3) / row.LFGA, axis=1) 

#Turnover Rate= Turnovers/(Field Goal Attempts + 0.44*Free Throw Attempts + Turnovers)
#he who doesnt turn the ball over wins games

df_season['WTOR'] = df_season.apply(lambda row: row.WTO / (row.WFGA + 0.44*row.WFTA + row.WTO), axis=1)
df_season['LTOR'] = df_season.apply(lambda row: row.LTO / (row.LFGA + 0.44*row.LFTA + row.LTO), axis=1)


#Offensive Rebounding Percentage = (Offensive Rebounds)/[(Offensive Rebounds)+(Opponent’s Defensive Rebounds)]
#You can win games controlling the offensive glass

df_season['WORP'] = df_season.apply(lambda row: row.WOR / (row.WOR + row.LDR), axis=1)
df_season['LORP'] = df_season.apply(lambda row: row.LOR / (row.LOR + row.WDR), axis=1)

#Free Throw Rate=(Free Throws Made)/(Field Goals Attempted) or Free Throws Attempted/Field Goals Attempted
#You got to get to the line to win close games

df_season['WFTAR'] = df_season.apply(lambda row: row.WFTA / row.WFGA, axis=1)
df_season['LFTAR'] = df_season.apply(lambda row: row.LFTA / row.LFGA, axis=1)

#4 Factors is weighted as follows
#1. Shooting (40%)
#2. Turnovers (25%)
#3. Rebounding (20%)
#4. Free Throws (15%)

df_season['W4Factor'] = df_season.apply(lambda row: .40*row.WeFGP + .25*row.WTOR + .20*row.WORP + .15*row.WFTAR, axis=1)
df_season['L4Factor'] = df_season.apply(lambda row: .40*row.LeFGP + .25*row.LTOR + .20*row.LORP + .15*row.LFTAR, axis=1)      

In [48]:
#Offensive efficiency (OffRtg) =  (Points / Possessions)
#Every possession counts

df_season['WOffRtg'] = df_season.apply(lambda row: (row.WScore / row.Possesions), axis=1)
df_season['LOffRtg'] = df_season.apply(lambda row: (row.LScore / row.Possesions), axis=1)

#Defensive efficiency (DefRtg) = (Opponent points / Opponent possessions)
#defense wins championships

df_season['WDefRtg'] = df_season.LOffRtg
df_season['LDefRtg'] = df_season.WOffRtg

                        
#Assist Ratio : Percentage of team possessions that end in assists
#distribute the rock - dont go isolation all the time

df_season['WAstR'] = df_season.apply(lambda row: row.WAst / (row.WFGA + 0.44*row.WFTA + row.WAst + row.WTO), axis=1)
df_season['LAstR'] = df_season.apply(lambda row: row.LAst / (row.LFGA + 0.44*row.LFTA + row.LAst + row.LTO), axis=1)


#DREB% : Percentage of team defensive rebounds
#control your own glass

df_season['WDRP'] = df_season.apply(lambda row: row.WDR / (row.WDR + row.LOR), axis=1)
df_season['LDRP'] = df_season.apply(lambda row: row.LDR / (row.LDR + row.WOR), axis=1) 

#Free Throw Percentage
#Make your damn free throws

df_season['WFTPCT'] = df_season.apply(lambda row : 0 if row.WFTA < 1 else row.WFTM / row.WFTA, axis=1)
df_season['LFTPCT'] = df_season.apply(lambda row : 0 if row.LFTA < 1 else row.LFTM / row.LFTA, axis=1)

In [49]:
df_season.drop(['WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR', 'WAst', 'WTO', 'WStl', 'WBlk', 'WPF'], axis=1, inplace=True)
df_season.drop(['LFGM', 'LFGA', 'LFGM3', 'LFGA3', 'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF'], axis=1, inplace=True)

In [50]:
df_season.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,Possesions,WPIE,LPIE,WeFGP,LeFGP,WTOR,LTOR,WORP,LORP,WFTAR,LFTAR,W4Factor,L4Factor,WOffRtg,LOffRtg,WDefRtg,LDefRtg,WAstR,LAstR,WDRP,LDRP,WFTPCT,LFTPCT
0,2003,10,1104,68,1328,62,N,0,69.888000,0.532847,0.467153,0.491379,0.433962,0.258659,0.223104,0.388889,0.294118,0.310345,0.415094,0.385546,0.350448,0.972985,0.887134,0.887134,0.972985,0.127551,0.090212,0.705882,0.611111,0.611111,0.727273
1,2003,10,1272,70,1393,63,N,0,65.356800,0.602339,0.397661,0.483871,0.402985,0.155950,0.136674,0.375000,0.416667,0.306452,0.298507,0.353504,0.323472,1.071044,0.963939,0.963939,1.071044,0.161031,0.073840,0.583333,0.625000,0.526316,0.450000
2,2003,11,1266,73,1437,61,N,0,61.382400,0.736434,0.263566,0.482759,0.321918,0.123824,0.126156,0.435897,0.543860,0.500000,0.315068,0.386239,0.316338,1.189266,0.993770,0.993770,1.189266,0.156642,0.086439,0.456140,0.564103,0.586207,0.608696
3,2003,11,1296,56,1457,50,N,0,55.315200,0.754717,0.245283,0.513158,0.428571,0.188561,0.254692,0.230769,0.472222,0.815789,0.306122,0.420926,0.375464,1.012380,0.903911,0.903911,1.012380,0.147374,0.107656,0.527778,0.769231,0.548387,0.533333
4,2003,11,1400,77,1208,71,N,0,60.768000,0.570732,0.429268,0.540984,0.435484,0.173439,0.119218,0.531250,0.488372,0.213115,0.435484,0.397970,0.366995,1.267114,1.168378,1.168378,1.267114,0.129422,0.125156,0.511628,0.468750,0.846154,0.629630


In [51]:
df_season_composite = pd.DataFrame()

#This will aggregate individual games into season totals for a team

#calculates wins and losses to get winning percentage

df_season_composite['WINS'] = df_season['WTeamID'].groupby([df_season['Season'], df_season['WTeamID']]).count()
df_season_composite['LOSSES'] = df_season['LTeamID'].groupby([df_season['Season'], df_season['LTeamID']]).count()
df_season_composite['WINPCT'] = df_season_composite['WINS'] / (df_season_composite['WINS'] + df_season_composite['LOSSES'])

# calculates averages for games team won

df_season_composite['WPIE'] = df_season['WPIE'].groupby([df_season['Season'], df_season['WTeamID']]).mean()
df_season_composite['WeFGP'] = df_season['WeFGP'].groupby([df_season['Season'], df_season['WTeamID']]).mean()
df_season_composite['WTOR'] = df_season['WTOR'].groupby([df_season['Season'], df_season['WTeamID']]).mean()
df_season_composite['WORP'] = df_season['WORP'].groupby([df_season['Season'], df_season['WTeamID']]).mean()
df_season_composite['WFTAR'] = df_season['WFTAR'].groupby([df_season['Season'], df_season['WTeamID']]).mean()
df_season_composite['W4Factor'] = df_season['W4Factor'].groupby([df_season['Season'], df_season['WTeamID']]).mean()
df_season_composite['WOffRtg'] = df_season['WOffRtg'].groupby([df_season['Season'], df_season['WTeamID']]).mean()
df_season_composite['WDefRtg'] = df_season['WDefRtg'].groupby([df_season['Season'], df_season['WTeamID']]).mean()
df_season_composite['WAstR'] = df_season['WAstR'].groupby([df_season['Season'], df_season['WTeamID']]).mean()
df_season_composite['WDRP'] = df_season['WDRP'].groupby([df_season['Season'], df_season['WTeamID']]).mean()
df_season_composite['WFTPCT'] = df_season['WFTPCT'].groupby([df_season['Season'], df_season['WTeamID']]).mean()

# calculates averages for games team lost

df_season_composite['LPIE'] = df_season['LPIE'].groupby([df_season['Season'], df_season['LTeamID']]).mean()
df_season_composite['LeFGP'] = df_season['LeFGP'].groupby([df_season['Season'], df_season['LTeamID']]).mean()
df_season_composite['LTOR'] = df_season['LTOR'].groupby([df_season['Season'], df_season['LTeamID']]).mean()
df_season_composite['LORP'] = df_season['LORP'].groupby([df_season['Season'], df_season['LTeamID']]).mean()
df_season_composite['LFTAR'] = df_season['LFTAR'].groupby([df_season['Season'], df_season['LTeamID']]).mean()
df_season_composite['L4Factor'] = df_season['L4Factor'].groupby([df_season['Season'], df_season['LTeamID']]).mean()
df_season_composite['LOffRtg'] = df_season['LOffRtg'].groupby([df_season['Season'], df_season['LTeamID']]).mean()
df_season_composite['LDefRtg'] = df_season['LDefRtg'].groupby([df_season['Season'], df_season['LTeamID']]).mean()
df_season_composite['LAstR'] = df_season['LAstR'].groupby([df_season['Season'], df_season['LTeamID']]).mean()
df_season_composite['LDRP'] = df_season['LDRP'].groupby([df_season['Season'], df_season['LTeamID']]).mean()
df_season_composite['LFTPCT'] = df_season['LFTPCT'].groupby([df_season['Season'], df_season['LTeamID']]).mean()

# calculates weighted average using winning percent to weight the statistic


df_season_composite['PIE'] = df_season_composite['WPIE'] * df_season_composite['WINPCT'] + df_season_composite['LPIE'] * (1 - df_season_composite['WINPCT'])
df_season_composite['FG_PCT'] = df_season_composite['WeFGP'] * df_season_composite['WINPCT'] + df_season_composite['LeFGP'] * (1 - df_season_composite['WINPCT'])
df_season_composite['TURNOVER_RATE'] = df_season_composite['WTOR'] * df_season_composite['WINPCT'] + df_season_composite['LTOR'] * (1 - df_season_composite['WINPCT'])
df_season_composite['OFF_REB_PCT'] = df_season_composite['WORP'] * df_season_composite['WINPCT'] + df_season_composite['LORP'] * (1 - df_season_composite['WINPCT'])
df_season_composite['FT_RATE'] = df_season_composite['WFTAR'] * df_season_composite['WINPCT'] + df_season_composite['LFTAR'] * (1 - df_season_composite['WINPCT'])
df_season_composite['4FACTOR'] = df_season_composite['W4Factor'] * df_season_composite['WINPCT'] + df_season_composite['L4Factor'] * (1 - df_season_composite['WINPCT'])
df_season_composite['OFF_EFF'] = df_season_composite['WOffRtg'] * df_season_composite['WINPCT'] + df_season_composite['LOffRtg'] * (1 - df_season_composite['WINPCT'])
df_season_composite['DEF_EFF'] = df_season_composite['WDefRtg'] * df_season_composite['WINPCT'] + df_season_composite['LDefRtg'] * (1 - df_season_composite['WINPCT'])
df_season_composite['ASSIST_RATIO'] = df_season_composite['WAstR'] * df_season_composite['WINPCT'] + df_season_composite['LAstR'] * (1 - df_season_composite['WINPCT'])
df_season_composite['DEF_REB_PCT'] = df_season_composite['WDRP'] * df_season_composite['WINPCT'] + df_season_composite['LDRP'] * (1 - df_season_composite['WINPCT'])
df_season_composite['FT_PCT'] = df_season_composite['WFTPCT'] * df_season_composite['WINPCT'] + df_season_composite['LFTPCT'] * (1 - df_season_composite['WINPCT'])

df_season_composite.reset_index(inplace = True)

In [52]:
#Kentucy and Witchita State went undefeated causing problems with the data since cant calculate average stats without WINPCT

df_season_composite[df_season_composite['LOSSES'].isnull()]

,Season,WTeamID,WINS,LOSSES,WINPCT,WPIE,WeFGP,WTOR,WORP,WFTAR,W4Factor,WOffRtg,WDefRtg,WAstR,WDRP,WFTPCT,LPIE,LeFGP,LTOR,LORP,LFTAR,L4Factor,LOffRtg,LDefRtg,LAstR,LDRP,LFTPCT,PIE,FG_PCT,TURNOVER_RATE,OFF_REB_PCT,FT_RATE,4FACTOR,OFF_EFF,DEF_EFF,ASSIST_RATIO,DEF_REB_PCT,FT_PCT
4064,2014,1455,33,nan,nan,0.697612,0.523191,0.137994,0.353977,0.479304,0.386466,1.222681,0.974460,0.149764,0.747270,0.731159,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
4211,2015,1246,34,nan,nan,0.809886,0.520134,0.137713,0.402745,0.451499,0.390756,1.221053,0.881479,0.158968,0.684470,0.719838,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [53]:
#Complete hack to fix the data

df_season_composite.loc[4064,'WINPCT'] = 1
df_season_composite.loc[4064,'LOSSES'] = 0
df_season_composite.loc[4064,'PIE'] = df_season_composite.loc[4064,'WPIE']
df_season_composite.loc[4064,'FG_PCT'] = df_season_composite.loc[4064,'WeFGP']
df_season_composite.loc[4064,'TURNOVER_RATE'] = df_season_composite.loc[4064,'WTOR']
df_season_composite.loc[4064,'OFF_REB_PCT'] = df_season_composite.loc[4064,'WORP']
df_season_composite.loc[4064,'FT_RATE'] = df_season_composite.loc[4064,'WFTAR']
df_season_composite.loc[4064,'4FACTOR'] = df_season_composite.loc[4064,'W4Factor']
df_season_composite.loc[4064,'OFF_EFF'] = df_season_composite.loc[4064,'WOffRtg']
df_season_composite.loc[4064,'DEF_EFF'] = df_season_composite.loc[4064,'WDefRtg']
df_season_composite.loc[4064,'ASSIST_RATIO'] = df_season_composite.loc[4064,'WAstR']
df_season_composite.loc[4064,'DEF_REB_PCT'] = df_season_composite.loc[4064,'WDRP']
df_season_composite.loc[4064,'FT_PCT'] = df_season_composite.loc[4064,'WFTPCT']

df_season_composite.loc[4211,'WINPCT'] = 1
df_season_composite.loc[4211,'LOSSES'] = 0
df_season_composite.loc[4211,'PIE'] = df_season_composite.loc[4211,'WPIE']
df_season_composite.loc[4211,'FG_PCT'] = df_season_composite.loc[4211,'WeFGP']
df_season_composite.loc[4211,'TURNOVER_RATE'] = df_season_composite.loc[4211,'WTOR']
df_season_composite.loc[4211,'OFF_REB_PCT'] = df_season_composite.loc[4211,'WORP']
df_season_composite.loc[4211,'FT_RATE'] = df_season_composite.loc[4211,'WFTAR']
df_season_composite.loc[4211,'4FACTOR'] = df_season_composite.loc[4211,'W4Factor']
df_season_composite.loc[4211,'OFF_EFF'] = df_season_composite.loc[4211,'WOffRtg']
df_season_composite.loc[4211,'DEF_EFF'] = df_season_composite.loc[4211,'WDefRtg']
df_season_composite.loc[4211,'ASSIST_RATIO'] = df_season_composite.loc[4211,'WAstR']
df_season_composite.loc[4211,'DEF_REB_PCT'] = df_season_composite.loc[4211,'WDRP']
df_season_composite.loc[4211,'FT_PCT'] = df_season_composite.loc[4211,'WFTPCT']

In [54]:
#we only need the final summary stats

df_season_composite.drop(['WINS','WPIE','WeFGP','WTOR','WORP','WFTAR','W4Factor','WOffRtg','WDefRtg','WAstR','WDRP','WFTPCT'], axis=1, inplace=True)
df_season_composite.drop(['LOSSES','LPIE','LeFGP','LTOR','LORP','LFTAR','L4Factor','LOffRtg','LDefRtg','LAstR','LDRP','LFTPCT'], axis=1, inplace=True)
df_season_composite.head()

,Season,WTeamID,WINPCT,PIE,FG_PCT,TURNOVER_RATE,OFF_REB_PCT,FT_RATE,4FACTOR,OFF_EFF,DEF_EFF,ASSIST_RATIO,DEF_REB_PCT,FT_PCT
0,2003,1102,0.428571,0.488599,0.584407,0.193121,0.168235,0.446693,0.382694,1.094137,1.091964,0.178061,0.630384,0.642402
1,2003,1103,0.481481,0.509717,0.536564,0.159805,0.305803,0.465135,0.385508,1.166466,1.163578,0.159196,0.626998,0.735271
2,2003,1104,0.607143,0.536514,0.475785,0.167275,0.371256,0.372350,0.362236,1.089621,1.029131,0.130983,0.686897,0.705168
3,2003,1105,0.269231,0.415050,0.457983,0.207792,0.335166,0.359501,0.356100,0.985661,1.054833,0.138288,0.641434,0.709598
4,2003,1106,0.464286,0.527568,0.481697,0.213721,0.349480,0.307563,0.362140,0.986712,0.991377,0.126527,0.679342,0.623158


In [55]:
#a little housekeeping to make easier to graph correlation matrix

columns = list(df_season_composite.columns.values) 
columns.pop(columns.index('WINPCT')) 
columns.append('WINPCT')
df_season_composite = df_season_composite[columns]
df_season_composite.rename(columns={'WTeamID':'TeamID'}, inplace=True)
df_season_composite.head()

,Season,TeamID,PIE,FG_PCT,TURNOVER_RATE,OFF_REB_PCT,FT_RATE,4FACTOR,OFF_EFF,DEF_EFF,ASSIST_RATIO,DEF_REB_PCT,FT_PCT,WINPCT
0,2003,1102,0.488599,0.584407,0.193121,0.168235,0.446693,0.382694,1.094137,1.091964,0.178061,0.630384,0.642402,0.428571
1,2003,1103,0.509717,0.536564,0.159805,0.305803,0.465135,0.385508,1.166466,1.163578,0.159196,0.626998,0.735271,0.481481
2,2003,1104,0.536514,0.475785,0.167275,0.371256,0.372350,0.362236,1.089621,1.029131,0.130983,0.686897,0.705168,0.607143
3,2003,1105,0.415050,0.457983,0.207792,0.335166,0.359501,0.356100,0.985661,1.054833,0.138288,0.641434,0.709598,0.269231
4,2003,1106,0.527568,0.481697,0.213721,0.349480,0.307563,0.362140,0.986712,0.991377,0.126527,0.679342,0.623158,0.464286


In [56]:
df_RPI = df_rankings[df_rankings['SystemName'] == 'RPI']
df_RPI_final = df_RPI[df_RPI['RankingDayNum'] == 133]
df_RPI_final.drop(labels=['RankingDayNum', 'SystemName'], inplace=True, axis=1)
df_RPI_final.head()

C:\Users\oakiyama\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


,Season,TeamID,OrdinalRank
127586,2003,1102,158
127587,2003,1103,182
127588,2003,1104,38
127589,2003,1105,313
127590,2003,1106,248


In [57]:
# Lets look at the conference tourney games to see if their is a correlation between a team doing well
# in the conference tourney and having that carry over to winning games in the big dance 

# Lets calculate the number of games a team played in the conference tournament

#df_ConferenceTourneyGames.head()
df_ConferenceTourneyGames = pd.read_csv('../input/ConferenceTourneyGames.csv')

ConferenceTourney_SummaryW = pd.DataFrame()
ConferenceTourney_SummaryL = pd.DataFrame()
ConferenceTourney_SummaryW['WINS'] = df_ConferenceTourneyGames['WTeamID'].groupby([df_ConferenceTourneyGames['Season'], df_ConferenceTourneyGames['WTeamID']]).count()
ConferenceTourney_SummaryL['LOSSES'] = df_ConferenceTourneyGames['LTeamID'].groupby([df_ConferenceTourneyGames['Season'], df_ConferenceTourneyGames['LTeamID']]).count()
##ConferenceTourney_SummaryW['LOSSES'].fillna(0, inplace=True)
#ConferenceTourney_SummaryW['GAMES'] = ConferenceTourney_SummaryW['WINS'] + ConferenceTourney_SummaryW['LOSSES']
ConferenceTourney_SummaryW.reset_index(inplace = True)
ConferenceTourney_SummaryL.reset_index(inplace = True)
ConferenceTourney_SummaryW.rename(columns={'WTeamID':'TeamID'}, inplace=True)
ConferenceTourney_SummaryL.rename(columns={'LTeamID':'TeamID'}, inplace=True)

ConferenceTourney_Summary = pd.merge(left=ConferenceTourney_SummaryW, right=ConferenceTourney_SummaryL, how='outer', on=['Season', 'TeamID'])
ConferenceTourney_Summary['LOSSES'].fillna(0, inplace=True) 
ConferenceTourney_Summary['GAMES'] = ConferenceTourney_Summary['LOSSES'] + ConferenceTourney_Summary['WINS']

ConferenceTourney_Summary.head()

,Season,TeamID,WINS,LOSSES,GAMES
0,2001.000000,1104.000000,1.000000,1.000000,2.000000
1,2001.000000,1106.000000,3.000000,0.000000,3.000000
2,2001.000000,1108.000000,2.000000,1.000000,3.000000
3,2001.000000,1111.000000,1.000000,1.000000,2.000000
4,2001.000000,1114.000000,1.000000,1.000000,2.000000


In [58]:
# Now lets do the same thing for the actual tourney

NCAATourney_SummaryW = pd.DataFrame()
NCAATourney_SummaryL = pd.DataFrame()
NCAATourney_SummaryW['WINS'] = df_tourney['WTeamID'].groupby([df_tourney['Season'], df_tourney['WTeamID']]).count()
NCAATourney_SummaryL['LOSSES'] = df_tourney['LTeamID'].groupby([df_tourney['Season'], df_tourney['LTeamID']]).count()
##NCAATourney_SummaryW['LOSSES'].fillna(0, inplace=True)
#NCAATourney_SummaryW['GAMES'] = NCAATourney_SummaryW['WINS'] + NCAATourney_SummaryW['LOSSES']
NCAATourney_SummaryW.reset_index(inplace = True)
NCAATourney_SummaryL.reset_index(inplace = True)
NCAATourney_SummaryW.rename(columns={'WTeamID':'TeamID'}, inplace=True)
NCAATourney_SummaryL.rename(columns={'LTeamID':'TeamID'}, inplace=True)

NCAATourney_Summary = pd.merge(left=NCAATourney_SummaryW, right=NCAATourney_SummaryL, how='outer', on=['Season', 'TeamID'])
NCAATourney_Summary['LOSSES'].fillna(0, inplace=True) 
NCAATourney_Summary['GAMES'] = NCAATourney_Summary['LOSSES'] + NCAATourney_Summary['WINS']

NCAATourney_Summary.head()

,Season,TeamID,WINS,LOSSES,GAMES
0,1985.000000,1104.000000,2.000000,1.000000,3.000000
1,1985.000000,1116.000000,1.000000,1.000000,2.000000
2,1985.000000,1120.000000,2.000000,1.000000,3.000000
3,1985.000000,1130.000000,2.000000,1.000000,3.000000
4,1985.000000,1181.000000,1.000000,1.000000,2.000000


In [59]:
Hot_team = pd.merge(left=NCAATourney_Summary, right=ConferenceTourney_Summary, how='left', on=['Season', 'TeamID'])

Hot_team['GAMES_y'].fillna(0, inplace=True) 
Hot_team.rename(columns={'GAMES_x':'Conf_Games'}, inplace=True)
Hot_team.rename(columns={'GAMES_y':'Tourney_Games'}, inplace=True)
Hot_team.drop(labels=['WINS_x', 'LOSSES_x', 'WINS_y', 'LOSSES_y'], inplace=True, axis=1)

Hot_team = Hot_team[Hot_team['Tourney_Games'] > 0]
Hot_team.iloc[:,2:4].corr()

,Conf_Games,Tourney_Games
Conf_Games,1.000000,0.048207
Tourney_Games,0.048207,1.000000


In [60]:
#Lets look at the team conferences file

df_team_conferences = df_team_conferences[df_team_conferences['Season'] > 2002]

df_team_conferences.head()

,Season,TeamID,ConfAbbrev
5407,2003,1125,a_sun
5408,2003,1144,a_sun
5409,2003,1194,a_sun
5410,2003,1205,a_sun
5411,2003,1209,a_sun


In [61]:
# For Strength of schedule measure we will take the conference average of RPI and assign it to each team in the conference
# All things being equal a team with a higher Conference RPI should win the game

df_team_conferences_stage = pd.merge(left=df_team_conferences, right=df_RPI_final, how='left', on=['Season', 'TeamID'])
df_team_conferences_stage.head(5)

,Season,TeamID,ConfAbbrev,OrdinalRank
0,2003,1125,a_sun,143.000000
1,2003,1144,a_sun,317.000000
2,2003,1194,a_sun,293.000000
3,2003,1205,a_sun,299.000000
4,2003,1209,a_sun,218.000000


In [62]:
#calculates the Conference RPI

df_team_conference_strength = pd.DataFrame()

df_team_conference_strength['Conf_Strength'] = df_team_conferences_stage['OrdinalRank'].groupby([df_team_conferences_stage['Season'], df_team_conferences_stage['ConfAbbrev']]).mean()
df_team_conference_strength.reset_index(inplace=True)
df_team_conference_strength.head()

,Season,ConfAbbrev,Conf_Strength
0,2003,a_sun,206.750000
1,2003,a_ten,128.583333
2,2003,acc,56.111111
3,2003,aec,217.111111
4,2003,big_east,77.214286


In [63]:
# Now we assign the Conference Strength back to each team

df_team_conferences_stage= pd.merge(left=df_team_conferences_stage, right=df_team_conference_strength, how='left', on=['Season', 'ConfAbbrev'])
df_team_conferences_stage.drop(labels=['OrdinalRank', 'ConfAbbrev'], inplace=True, axis=1)

df_team_conferences_stage.head()

,Season,TeamID,Conf_Strength
0,2003,1125,206.750000
1,2003,1144,206.750000
2,2003,1194,206.750000
3,2003,1205,206.750000
4,2003,1209,206.750000


In [64]:
#Get seeds of teams for all tourney games

df_seeds.head()

,Season,Seed,TeamID
0,1985,W01,1207
1,1985,W02,1210
2,1985,W03,1228
3,1985,W04,1260
4,1985,W05,1374


In [65]:
# Convert string to an integer

df_seeds['seed_int'] = df_seeds['Seed'].apply( lambda x : int(x[1:3]) )
df_seeds.drop(labels=['Seed'], inplace=True, axis=1) 
df_seeds.rename(columns={'seed_int':'Seed'},inplace=True)

# Merge in the Conference Strength variable here

df_seeds= pd.merge(left=df_seeds, right=df_team_conferences_stage, how='left', on=['Season', 'TeamID'])

df_seeds.head()

,Season,TeamID,Seed,Conf_Strength
0,1985,1207,1,nan
1,1985,1210,2,nan
2,1985,1228,3,nan
3,1985,1260,4,nan
4,1985,1374,5,nan
